In [1]:
import os
from dotenv import load_dotenv

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

load_dotenv()

True

In [2]:
CREDENTIALS_PATH = os.getenv("CREDENTIALS_PATH")

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "../lib/gcs-connector-hadoop3-2.2.15.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", CREDENTIALS_PATH)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", CREDENTIALS_PATH)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
SPARK_GCS_DATA = os.getenv("SPARK_GCS_DATA")

In [6]:
df = spark.read.parquet(f'{SPARK_GCS_DATA}')

In [7]:
df.show(truncate=True)

+--------------------+----------------------------+-------------------+-------------+--------------------+--------------------+--------------------+---------------------------------+-----------------------+--------------------+---------------------------------+------------------------------------------+--------------+--------------------+-----------------+----------+----------+
|       Hotel_Address|Additional_Number_of_Scoring|        Review_Date|Average_Score|          Hotel_Name|Reviewer_Nationality|     Negative_Review|Review_Total_Negative_Word_Counts|Total_Number_of_Reviews|     Positive_Review|Review_Total_Positive_Word_Counts|Total_Number_of_Reviews_Reviewer_Has_Given|Reviewer_Score|                Tags|days_since_review|       lat|       lng|
+--------------------+----------------------------+-------------------+-------------+--------------------+--------------------+--------------------+---------------------------------+-----------------------+--------------------+-----------

In [9]:
# spark.stop()